# Classification of Sugarcane Diseases based on Images

## Initial Setup

Examining the train data shows that there are six (6) classes in total:

In [45]:
classes = [
    "Banded_Chlorosis",
    "Brown_Rust",
    "Brown_Spot",
    "Viral",
    "Yellow_Leaf",
    "Healthy",
]

To make it easier to handle the image data, a Python class called `Classification` is defined and it will use the OpenCV library.

This class will contain the method `load_images` that can be used to load the images, as well as the method `resize_images` for resizing all the images.

Property methods `image_count` and `image_dimensions` also allow us to analyze the loaded images and determine if further preprocessing is needed.

In [46]:
from pathlib import Path

import cv2
import numpy as np


class Classification:
    name: str
    images: list[np.ndarray]

    def __init__(self, name: str):
        self.name = name
        self.images = list()

    @property
    def image_count(self) -> int:
        return len(self.images)

    @property
    def image_dimensions(self) -> dict[tuple[int, int, int], int]:
        dims = {}
        for img in self.images:
            if dims.get(img.shape) is None:
                dims[img.shape] = 1
                continue
            dims[img.shape] += 1
        return dims

    @property
    def image_dimensions_distribution(self) -> dict[tuple[int, int, int], int]:
        distrib = {}
        for key, value in self.image_dimensions.items():
            distrib[key] = value / self.image_count
        return distrib

    def load_images(self, top_folder: str = "train") -> None:
        """
        Empties self.images  then loads images using opencv imread

        Returns the length of image_array

        Raises exception upon error
        """
        try:
            self.images = list()
            path = Path(f"{top_folder}/{self.name}")
            image_paths = [img_path for img_path in path.iterdir() if img_path.is_file()]
            for path in image_paths:
                self.images.append(cv2.imread(str(path)))

        except Exception as e:
            raise e

    def square_images(self, image_size: int = 512, with_padding: bool = True) -> None:
        """
        Resizes all images in self.image_array to the specified image size.

        In the event that the source image is not a square and with_padding is True,
        padding will be added to the smaller side to ensure aspect ratio of 1.0.

        Returns None
        """
        for index, item in enumerate(self.images):
            height, width, _ = self.images[index].shape
            aspect_ratio = height / width

            if not with_padding or aspect_ratio == 1:
                self.images[index] = cv2.resize(item, (image_size, image_size))
                continue

            padding_color = (0, 0, 0)
            if aspect_ratio > 1:
                width_padding = (height - width) // 2
                image = cv2.copyMakeBorder(
                    item,
                    0,
                    0,
                    width_padding,
                    width_padding,
                    cv2.BORDER_CONSTANT,
                    value=padding_color,
                )

            else:
                height_padding = (width - height) // 2
                image = cv2.copyMakeBorder(
                    item,
                    height_padding,
                    height_padding,
                    0,
                    0,
                    cv2.BORDER_CONSTANT,
                    value=padding_color,
                )

            self.images[index] = cv2.resize(image, (image_size, image_size))

    def normalize_images(self) -> None:
        """
        Normalizes the values to be from 0 to 1 instead of 0 to 255.

        Returns None
        """
        for index, item in enumerate(self.images):
            self.images[index] = item / 255.0

    def __repr__(self) -> str:
        return f"{self.name}\n image count: {self.image_count}\n image dimensions: {self.image_dimensions}\n dimension distribution: {self.image_dimensions_distribution}"

## Data Loading and Preprocessing

First, create the instances for each class.

In [47]:
classifications = [Classification(_class) for _class in classes]

Then, for each instance, let us load the images under the path `./train/<class_name>`.

We will also display the number of images as well as the dimensions of all images.

In [48]:
for x in classifications:
    x.load_images()
    print(x)

Banded_Chlorosis
 image count: 424
 image dimensions: {(1024, 768, 3): 404, (576, 768, 3): 20}
 dimension distribution: {(1024, 768, 3): 0.9528301886792453, (576, 768, 3): 0.04716981132075472}
Brown_Rust
 image count: 282
 image dimensions: {(1024, 768, 3): 280, (576, 768, 3): 2}
 dimension distribution: {(1024, 768, 3): 0.9929078014184397, (576, 768, 3): 0.0070921985815602835}
Brown_Spot
 image count: 1550
 image dimensions: {(1024, 768, 3): 1481, (576, 768, 3): 69}
 dimension distribution: {(1024, 768, 3): 0.9554838709677419, (576, 768, 3): 0.044516129032258066}
Viral
 image count: 597
 image dimensions: {(1024, 768, 3): 501, (576, 768, 3): 96}
 dimension distribution: {(1024, 768, 3): 0.8391959798994975, (576, 768, 3): 0.16080402010050251}
Yellow_Leaf
 image count: 1074
 image dimensions: {(1024, 768, 3): 1005, (576, 768, 3): 69}
 dimension distribution: {(1024, 768, 3): 0.9357541899441341, (576, 768, 3): 0.06424581005586592}
Healthy
 image count: 387
 image dimensions: {(1024, 768,

Two important pieces of information can be gleaned from the output.

1. It can be seen that the six (6) different classes have an imbalance in image count.

This is a problem because it may introduce excessive bias in our models.
To avoid this problem, when training the models, we must use proper sampling techniques to ensure equal class distribution.

2. The images are rectangular and the dimensions are not homogenous.

Most images have a size of `1024x768` but there are some whose size are `576x768` instead.
To ensure that our model will be able to process these images later, we can pad the images to make them a square, and then we resize it.

Lastly, let us normalize the data so that the values are from 0 to 1 instead of 0 to 255.

In [ ]:
# Resize all images to 512x512
IMAGE_SIZE = 128
for x in classifications:
    x.square_images(128, with_padding=True)
    x.normalize_images()
    print(x)

Banded_Chlorosis
 image count: 424
 image dimensions: {(512, 512, 3): 424}
 dimension distribution: {(512, 512, 3): 1.0}
Brown_Rust
 image count: 282
 image dimensions: {(512, 512, 3): 282}
 dimension distribution: {(512, 512, 3): 1.0}
Brown_Spot
 image count: 1550
 image dimensions: {(512, 512, 3): 1550}
 dimension distribution: {(512, 512, 3): 1.0}
Viral
 image count: 597
 image dimensions: {(512, 512, 3): 597}
 dimension distribution: {(512, 512, 3): 1.0}
Yellow_Leaf
 image count: 1074
 image dimensions: {(512, 512, 3): 1074}
 dimension distribution: {(512, 512, 3): 1.0}
Healthy
 image count: 387
 image dimensions: {(512, 512, 3): 387}
 dimension distribution: {(512, 512, 3): 1.0}


With the images resized, we must also transform our categorical features into a numeric array.

This can be done using Scikit-learn's LabelBinarizer class.

In [87]:
from sklearn.preprocessing import LabelBinarizer

binarizer = LabelBinarizer()
class_labels: list[np.array] = binarizer.fit_transform(classes)  # type: ignore
label_pairings: dict[str, np.ndarray] = dict(zip(classes, class_labels, strict=False))
# Print output and classes to verify
print(f"Binarizer classes: {binarizer.classes_}")
print(label_pairings)

Binarizer classes: ['Banded_Chlorosis' 'Brown_Rust' 'Brown_Spot' 'Healthy' 'Viral'
 'Yellow_Leaf']
{'Banded_Chlorosis': array([1, 0, 0, 0, 0, 0]), 'Brown_Rust': array([0, 1, 0, 0, 0, 0]), 'Brown_Spot': array([0, 0, 1, 0, 0, 0]), 'Viral': array([0, 0, 0, 0, 1, 0]), 'Yellow_Leaf': array([0, 0, 0, 0, 0, 1]), 'Healthy': array([0, 0, 0, 1, 0, 0])}


With our processed images and our transformed class labels, let use generate the arrays which will serve as our input array `X` and label array `Y`.

This can be accomplished by combining the images of our classes and generating the corresponding label array using `label_pairings`.

In [88]:
# Create our array X, the input array
X: np.ndarray = np.vstack(
    [np.array(class_.images) for class_ in classifications]
)  # convert the image list of each class into an array then combine them

# Create our array Y, the label array
Y: np.ndarray = np.vstack(
    [
        [label_pairings[class_.name] for _ in range(class_.image_count)]
        for class_ in classifications
    ],
)  # generate the list of one-hot encoded vectors for each class then combine them

print(f"X.shape: {X.shape}\nY.shape:{Y.shape}")

X.shape: (4314, 128, 128, 3)
Y.shape:(4314, 6)



However, we must not forget that our classes are imbalanced.

It is important that balance this out to avoid overfitting towards the classes with a much larger count.

To balance our classes, we can used the Imbalanced-Learn library and then undersample the Majority class.

In [89]:
from imblearn.under_sampling import RandomUnderSampler

RANDOM_STATE_INT = 1738  # Seed for consistency
sampler = RandomUnderSampler(random_state=RANDOM_STATE_INT, replacement=False)

X_flattened = X.reshape(X.shape[0], -1)  # Flatten X for compatibility with RandomUnderSampler

X_balanced: np.ndarray  # type annotation
Y_balanced: np.ndarray  # type annotation
X_balanced, Y_balanced = sampler.fit_resample(X_flattened, Y)  # type: ignore

X_balanced = X_balanced.reshape(-1, *X.shape[1:])
print(f"X_balanced.shape: {X_balanced.shape}\nY_balanced.shape:{Y_balanced.shape}")

X_balanced.shape: (1692, 128, 128, 3)
Y_balanced.shape:(1692, 6)


`X_balanced` and `Y_balanced` represent our data with balanced samples.

This can be verified by the fact that the number of elements `1692` divided by our number of classes `6` results in `282`, which is the cardinality of our minority class, `Brown_Rust` according to our previous cells.

Lastly, for validation purposes of our model, a train-test split will be conducted on our data. 

In [90]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X_balanced, Y_balanced, train_size=0.8, random_state=RANDOM_STATE_INT, shuffle=True
)

print(
    f"Shapes\n X_train: {len(X_train)}\n Y_train: {len(Y_train)}\n X_test: {len(X_test)}\n Y_test: {len(Y_test)}"
)

Shapes
 X_train: 1353
 Y_train: 1353
 X_test: 339
 Y_test: 339


## Method 1: Convolution Neural Network (CNN)

The first method for solving this classification problem is through the use of CNN.

We will be using the Keras and TensorFlow libraries.

First, let us setup the environment and model.

In [ ]:
import os

import keras

# Set Keras backend to use TensorFlow
os.environ["KERAS_BACKEND"] = "tensorflow"

# Define the expected inputs
image_inputs = keras.Input(
    shape=classifications[0].images[0].shape, dtype=classifications[0].images[0].dtype
)

# Define model

float64
